# M-F

In [3]:
import os
import numpy as np
from scipy.ndimage import gaussian_filter, uniform_filter, median_filter
import rasterio
from scipy.ndimage import generic_filter
from scipy.stats import mode

def read_raster(file_path):
    with rasterio.open(file_path) as src:
        data = src.read(1)
        profile = src.profile
    return data, profile

def save_raster(data, profile, output_path):
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(data, 1)


# def apply_mean_filter(data, size=3):
#     filtered_data = uniform_filter(data, size=size)
#     filtered_data = np.round(filtered_data).astype(int)
#     filtered_data = np.clip(filtered_data, 0, 6)
#     return filtered_data

def apply_mode_filter(data, size=3):
    """
    Applies plurality filtering to the input 2D data and handles the boundary case, the boundary fill value (-1) is not involved in the calculation.
    
    Parameters:
    - data: input data (two-dimensional array)
    - size: size of the filter window (default is 3, i.e. 3x3 window)
    
    return:
    - The data after the multinomial filtering
    """
    # Getting the shape of the data
    rows, cols = data.shape
    
    # Fill the data at the boundary, the filling method can be chosen according to the demand, here use 'constant' to fill -1
    padded_data = np.pad(data, pad_width=size//2, mode='constant', constant_values=-1)
    
    # Create a new array to store the filtered data
    filtered_data = np.zeros_like(data)
    
    # Iterate over each element of the data array (skip boundaries)
    for i in range(rows):
        for j in range(cols):
            # Get the range of the window (using populated data)
            row_min = i
            row_max = i + size
            col_min = j
            col_max = j + size
            
            # Extract the value in the window (from the populated data)
            window = padded_data[row_min:row_max, col_min:col_max]
            
            # Remove elements with a value of -1
            window = window[window != -1]
            
            # Calculate the plural if there are valid values in the window
            if window.size > 0:
                filtered_data[i, j] = mode(window, axis=None).mode
            else:
                # If there is no valid value, then you can choose the default value, which is set here to -1
                filtered_data[i, j] = -1
    
    return filtered_data

# Get the directory where the current script is located
base_path = os.path.abspath('')  

# Define image paths and output paths
images_paths = [
    os.path.join(base_path, 'data', 'PITSCM', 'TS_2020_RF.tif'),
]

output_paths = os.path.join(base_path, 'data', 'M-F', 'TS_2020_RF_M-F.tif')

# Read raster data
data, profile = read_raster(images_paths[0])

import time
# Record code start time
start_time = time.time()
# Apply M-F and save
mean_filtered_data = apply_mode_filter(data, size=3)

# Record code end time
end_time = time.time()
# Output runtime
print(f"Code Runtime: {end_time - start_time:.4f}s")
save_raster(mean_filtered_data, profile, output_paths)


C:\Users\admin 1\AppData\Local\Temp\ipykernel_50596\2501757353.py:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  filtered_data[i, j] = mode(window, axis=None).mode


Code Runtime: 1.6624s
